In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
import math
import numpy as np
from config import seq_len, batch_size, d_model, vocab_size
from utils import tokenize, clean, encode_position

In [ ]:
clean('../data/raw_corpus.txt', '../data/clean_corpus.txt')

with open('../data/clean_corpus.txt', 'r', encoding='utf-8') as f:
    corpus = f.read()

tokenize(corpus)

In [ ]:
tokenized_corpus = np.load('../data/tokenized_corpus_ids.npy')

sequences = len(tokenized_corpus) - seq_len

# Make training examples with stride = 1
x = np.array([tokenized_corpus[i:i+seq_len] for i in range(sequences)])
y = np.array([tokenized_corpus[i+1:i+seq_len+1] for i in range(sequences)])

batches = len(x) // batch_size

x_batches = np.array_split(x[:batches*batch_size], batches)
y_batches = np.array_split(y[:batches*batch_size], batches)

# Consider using Xavier initialization here
embedding_factor = 1 / math.sqrt(d_model)  # Scale factor for embedding matrix
embedding_matrix = np.random.randn(vocab_size, d_model) * embedding_factor

# Example: 0th batch
# embedded_batch = embedding_matrix[x_batches[0]]
# encode_position(embedded_batch)

# Attention
## Initialize weights (Xavier normal)
def xavier_normal(fan_in, fan_out, shape=None):
    if shape is None:
        shape = (fan_in, fan_out)

    std = np.sqrt(2 / (fan_in + fan_out))

    return np.random.normal(loc=0, scale=std, size=shape)

W_Q = xavier_normal(d_model, d_model)
W_K = xavier_normal(d_model, d_model)
W_V = xavier_normal(d_model, d_model)



[[-8.80616786e-02 -4.39130152e-03  7.72558969e-03 ...  1.46430895e-01
  -8.41336007e-03  5.85868499e-02]
 [-6.75439550e-02  6.41043421e-02 -9.61822785e-02 ... -4.23195761e-02
   6.18879602e-02  3.15545277e-02]
 [ 1.98467025e-02 -8.78545684e-02 -6.16623535e-02 ... -8.88480949e-02
  -1.32007605e-02 -4.07274055e-03]
 ...
 [-1.20466581e-02 -9.36947741e-02  8.05500370e-02 ...  2.17935640e-01
  -3.45430508e-02 -1.12319345e-01]
 [-1.22404076e-01  2.14786922e-02  5.13239750e-02 ...  8.86542853e-05
  -8.15786489e-02  1.30526526e-01]
 [ 6.10368223e-02 -9.32043901e-02  3.44309413e-03 ... -1.18576939e-01
   9.18917164e-02  4.61330598e-02]] (128, 128)
